<a href="https://colab.research.google.com/github/Green0v0/HpAnalytics/blob/main/Mini_proj_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix, log_loss, plot_roc_curve, auc, precision_recall_curve
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline
from xgboost import XGBClassifier
from functools import partial

from sklearn.model_selection import train_test_split, GridSearchCV
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My Drive/Colab Notebooks/dataset/input

In [ ]:
train= pd.read_csv('aug_train.csv')
test = pd.read_csv('aug_test.csv')
target = np.load('jobchange_test_target_values.npy')

In [ ]:
test['target'] = -1 #remeber that we have to drop this column later

df_pre = pd.concat([train, test], axis = 0).reset_index(drop = True)

In [ ]:
lb_en = LabelEncoder()

df_pre['city'] = lb_en.fit_transform(df_pre.loc[:,'city']) 

In [ ]:
df_pre['education_level'].value_counts(dropna=False)

In [ ]:
education_level = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
for i, v in enumerate(education_level):
    df_pre.loc[df_pre['education_level']==v, 'education_level'] = i

In [ ]:
df_pre['education_level'].value_counts(dropna=False)

In [ ]:
df_pre['last_new_job'].value_counts(dropna=False)

In [ ]:
df_pre.loc[df_pre['last_new_job'] == '>4', 'last_new_job'] = 5
df_pre.loc[df_pre['last_new_job'] == 'never', 'last_new_job'] = 0

df_pre['last_new_job'] = pd.to_numeric(df_pre['last_new_job'], downcast='integer')

In [ ]:
df_pre['last_new_job'].value_counts(dropna=False)

In [ ]:
df_pre.loc[df_pre['experience'] == '>20', 'experience'] = 21
df_pre.loc[df_pre['experience'] == '<1', 'experience'] = 0

df_pre['experience'] = pd.to_numeric(df_pre['experience'], downcast='integer')

In [ ]:
df_pre['experience'].value_counts(dropna=False)

In [ ]:
df_pre['enrolled_university'].value_counts(dropna=False)

In [ ]:
df_pre.loc[df_pre['enrolled_university'] == 'Full time course', 'enrolled_university'] = 2
df_pre.loc[df_pre['enrolled_university'] == 'Part time course', 'enrolled_university'] = 1
df_pre.loc[df_pre['enrolled_university'] == 'no_enrollment', 'enrolled_university'] = 0

In [ ]:
df_pre['enrolled_university'].value_counts(dropna=False)

In [ ]:
df_pre.isna().sum()

In [ ]:
gender_limit = df_pre['gender'].isna().sum()//2

In [ ]:
df_pre['gender'].fillna('Female', limit=gender_limit, inplace=True)
df_pre['gender'].fillna('Male', inplace=True)

In [ ]:
limit = df_pre.shape[0] * 0.03

In [ ]:
under_list = []
for col in df_pre.columns:
    if 1 < df_pre[col].isna().sum() < limit:
        under_list.append(col)
under_list

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors = 1)
df_pre[under_list] = imputer.fit_transform(df_pre[under_list])

In [ ]:
df_pre['major_discipline'].value_counts()

In [ ]:
major = df_pre['major_discipline'].value_counts().idxmax()
major

In [ ]:
df_pre['major_discipline'].fillna(major, inplace=True)

In [ ]:
df_pre.loc[df_pre['major_discipline'] == 'STEM', 'major_discipline'] = 'major'
df_pre.loc[df_pre['major_discipline'] != 'major', 'major_discipline'] = 'no_major'

In [ ]:
df_pre['major_discipline'].value_counts(dropna=False)

In [ ]:
df_pre['company_size'].fillna('unknown', inplace=True)
df_pre['company_type'].fillna('unknown', inplace=True)

In [ ]:
df_pre.isna().sum()

In [ ]:
df_pre.head()

In [ ]:
gender_map = {
        'Female': 2,
        'Male': 1,
        'Other': 0
         }

relevent_experience_map = {
    'Has relevent experience':  1,
    'No relevent experience' :  0
}


major_map ={ 
    'major'                   :    1,
    'no_major'        :    0
}
    

    
company_type_map = {
    'Pvt Ltd'               :    0,
    'Funded Startup'        :    1, 
    'Early Stage Startup'   :    2, 
    'Other'                 :    3, 
    'Public Sector'         :    4, 
    'NGO'                   :    5,
        'unknown'      :    6,

}

company_size_map = {
    '<10'          :    0,
    '10/49'        :    1, 
    '100-500'      :    2, 
    '1000-4999'    :    3, 
    '10000+'       :    4, 
    '50-99'        :    5, 
    '500-999'      :    6, 
    '5000-9999'    :    7,
    'unknown'      :    8,
}
    


In [ ]:
df_pre['company_size'] = df_pre['company_size'].map(company_size_map)
df_pre['company_type'] = df_pre['company_type'].map(company_type_map)
df_pre['major_discipline'] = df_pre['major_discipline'].map(major_map)
df_pre['relevent_experience'] = df_pre['relevent_experience'].map(relevent_experience_map)
df_pre['gender'] = df_pre['gender'].map(gender_map)


In [ ]:
df_pre.info()

In [ ]:
category_feature = ['gender', 'relevent_experience', 
                    'major_discipline', 'company_size', 'company_type']

numeric_feature = ['city','city_development_index', 'training_hours', 
'education_level', 'last_new_job',  'experience', 'enrolled_university']

df_numeric_feature = df_pre[numeric_feature]
df_categorical = df_pre[category_feature]
df_categorical.head()                   

In [ ]:
ohe = OneHotEncoder()
X = ohe.fit_transform(df_categorical)
ohe.get_feature_names()

In [ ]:
X = X.toarray()
X_df = pd.DataFrame(X, columns=ohe.get_feature_names())
X_df.shape

In [ ]:
df_numeric_feature.shape

In [ ]:
df = pd.concat([X_df,df_numeric_feature,df_pre['target']],axis = 1).reset_index(drop = True)

df.rename(columns = {"x3_<10": "x3_10-"}, inplace=True)
df.columns

In [ ]:
train = df_pre[df_pre['target'] != -1].reset_index(drop = True)
test = df_pre[df_pre['target'] == -1].reset_index(drop = True)

test = test.drop(['enrollee_id','target'], axis = 1)

X = train.drop(['enrollee_id', 'target'], axis = 1)
Y = train['target']
X.shape, Y.shape

In [ ]:
test.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2 , stratify = Y, shuffle=True, random_state = 42)

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('TREE', DecisionTreeClassifier(random_state = 42)))
models.append(('Random Forest', RandomForestClassifier(random_state = 42)))
models.append(('SVM', SVC(gamma='auto', random_state = 42)))
models.append(('XGBoost', XGBClassifier(random_state = 42)))
models.append(('GradientBoosting', GradientBoostingClassifier(random_state = 42)))

# evaluate each model in turn
results = []
names = []

In [ ]:
clf = XGBClassifier(learning_rate=0.02, n_estimators=300,objective='binary:logistic')

xgb_param_grid = {'max_depth':[3,5,6,8,10],
                  'subsample': [0.6, 0.8, 1.0],}


hr_grid = GridSearchCV(estimator=clf,
                       param_grid=xgb_param_grid,
                       scoring='accuracy',
                       n_jobs=-1,
                       cv=5,
                       refit=True, 
                       return_train_score=True)

hr_grid.fit(X_train, y_train,eval_metric=["error", "logloss"],
            eval_set = [(X_train, y_train), (X_test, y_test)], verbose=0)

In [ ]:
print(hr_grid.best_estimator_)
print(hr_grid.best_params_)
print(hr_grid.best_score_)

In [ ]:
Model = XGBClassifier(learning_rate=0.02, n_estimators=300,
                    objective='binary:logistic',max_depth = hr_grid.best_params_['max_depth'], subsample = hr_grid.best_params_['subsample'])
Model.fit(X_train,y_train,eval_metric=["error", "logloss"],
            eval_set = [(X_train, y_train), (X_test, y_test)],verbose=0)

In [ ]:
y_pred_t = Model.predict(X_train)
y_pred = Model.predict(X_test)

accuracy_train = accuracy_score(y_train, y_pred_t)
accuracy_test = accuracy_score(y_test, y_pred)

print("Accuracy: %.2f%%" % (accuracy_train * 100.0))
print("Accuracy: %.2f%%" % (accuracy_test * 100.0))
print(classification_report(y_test,y_pred))

In [ ]:
results = Model.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots(figsize=(12,7))
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_os, Y_os = sm.fit_resample(X,Y)

sns.countplot(Y_os, edgecolor = 'black')

X_train, X_test, y_train, y_test = train_test_split(X_os, Y_os, test_size = 0.2, stratify = Y_os, shuffle=True,random_state = 42)

In [ ]:
X_train.shape

In [ ]:
clf_os = XGBClassifier()
clf_os.fit(X_train, y_train,eval_metric=["error", "logloss"],
             eval_set = [(X_train, y_train), (X_test, y_test)],verbose = 0)

y_pred_t = clf_os.predict(X_train)
y_pred = clf_os.predict(X_test)

accuracy_train = accuracy_score(y_train, y_pred_t)
accuracy_test = accuracy_score(y_test, y_pred)

print("Accuracy: %.2f%%" % (accuracy_train * 100.0))
print("Accuracy: %.2f%%" % (accuracy_test * 100.0))
print(classification_report(y_test,y_pred))

In [ ]:
clf_os = XGBClassifier(learning_rate=0.02, n_estimators=300,objective='binary:logistic')

xgb_param_grid = {'max_depth':[3,5,6,8,10],
                  'subsample': [0.6, 0.8, 1.0],}


hr_grid = GridSearchCV(estimator=clf_os,
                       param_grid=xgb_param_grid,
                       scoring='accuracy',
                       n_jobs=-1,
                       cv=5,
                       refit=True, 
                       return_train_score=True)

hr_grid.fit(X_train, y_train,eval_metric=["error", "logloss"],verbose = 0)

In [ ]:
print(hr_grid.best_estimator_)
print(hr_grid.best_params_)
print(hr_grid.best_score_)

In [ ]:
# {'max_depth': 10, 'subsample': 0.8}
Model = XGBClassifier(learning_rate=0.02, n_estimators=300, max_depth = hr_grid.best_params_['max_depth'],
                      subsample = hr_grid.best_params_['subsample'],objective='binary:logistic')
Model.fit(X_train,y_train,eval_metric=["error", "logloss"],
            eval_set = [(X_train, y_train), (X_test, y_test)],verbose=0)


In [ ]:
y_pred_t = Model.predict(X_train)
y_pred = Model.predict(X_test)

accuracy_train = accuracy_score(y_train, y_pred_t)
accuracy_test = accuracy_score(y_test, y_pred)

print("Accuracy: %.2f%%" % (accuracy_train * 100.0))
print("Accuracy: %.2f%%" % (accuracy_test * 100.0))
print(classification_report(y_test,y_pred))

In [ ]:
roc_auc_score_train = roc_auc_score(y_train, y_pred_t)
roc_auc_score_test = roc_auc_score(y_test, y_pred)

print("Accuracy: %.2f%%" % (roc_auc_score_train * 100.0))
print("Accuracy: %.2f%%" % (roc_auc_score_test * 100.0))

In [ ]:
test_n = test.to_numpy()

y_pred = Model.predict(test_n)
accuracy_test = accuracy_score(y_pred, target)
print("Accuracy: %.2f%%" % (accuracy_test * 100.0))

In [ ]:
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import TomekLinks

undersample  = TomekLinks(random_state=42)
X_us, Y_us = undersample.fit_resample(X, Y)

# X_us, X_us = TomekLinks().fit_sample(X_imb, y_imb)  


sns.countplot(Y_us, edgecolor = 'black')

X_train, X_test, y_train, y_test = train_test_split(X_us, Y_us, test_size = 0.2, stratify = Y_us, shuffle=True,random_state = 42)

In [ ]:
clf_us = XGBClassifier(learning_rate=0.02, n_estimators=300,objective='binary:logistic')

xgb_param_grid = {'max_depth':[3,5,6,8,10],
                  'subsample': [0.6, 0.8, 1.0],}


hr_grid = GridSearchCV(estimator=clf_us,
                       param_grid=xgb_param_grid,
                       scoring='accuracy',
                       n_jobs=-1,
                       cv=5,
                       refit=True, 
                       return_train_score=True)

hr_grid.fit(X_train, y_train, eval_metric=["error", "logloss"],
            eval_set = [(X_train, y_train), (X_test, y_test)],verbose = 0)

In [ ]:
print(hr_grid.best_estimator_)
print(hr_grid.best_params_)
print(hr_grid.best_score_)

In [ ]:
#{'max_depth': 5, 'subsample': 1.0}
Model = XGBClassifier(learning_rate=0.02, n_estimators=300, max_depth = hr_grid.best_params_['max_depth'],
                      subsample = hr_grid.best_params_['subsample'],objective='binary:logistic')
Model.fit(X_train,y_train,eval_metric=["error", "logloss"],
            eval_set = [(X_train, y_train), (X_test, y_test)],verbose=0)

In [ ]:
y_pred_t = Model.predict(X_train)
y_pred = Model.predict(X_test)

accuracy_train = accuracy_score(y_train, y_pred_t)
accuracy_test = accuracy_score(y_test, y_pred)

print("Accuracy: %.2f%%" % (accuracy_train * 100.0))
print("Accuracy: %.2f%%" % (accuracy_test * 100.0))
print(classification_report(y_test,y_pred))